In [78]:
import pandas as pd
import numpy as np
week1_raw = pd.read_csv("week1_results.csv")
#week2_raw = pd.read_csv("week2_results.csv)
#week1_raw.head()


In [79]:
#function to combine picks and point values
def combine(df1):
    my_file = df1
    rows = my_file.shape[0]
    i = 2 
    while i < rows:
        for col in my_file.loc[:, my_file.columns != 'Name']:
            my_file[col][i+1] = float(my_file[col][i+1])
            my_file[col][i] = [my_file[col][i], my_file[col][i+1]]
        i = i+2
    return my_file

week1 = combine(week1_raw)

In [80]:
#drop unneeded rows
week1.drop([5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41, 42, 43], axis=0, inplace=True)
week1.tail()

#i = 3
#rows = week1.shape[1]
#while i < 6:
#    print(week1.iloc[])
#    i = i + 2

,Name,Game 1,Game 2,Game 3,Game 4,Game 5,Game 6,Game 7,Game 8,Game 9,Game 10,Game 11,Game 12,Game 13,Game 14,Game 15,Game 16
32,taz7611,"[LAR, 3.0]","[ATL, 2.0]","[BAL, 15.0]","[CIN, 16.0]","[MIA, 6.0]","[WAS, 13.0]","[CHI, 1.0]","[CAR, 11.0]","[IND, 12.0]","[PHI, 8.0]","[LAC, 7.0]","[GB, 10.0]","[TEN, 14.0]","[KC, 4.0]","[TB, 5.0]","[DEN, 9.0]"
34,Team-Boylan,"[BUF, 1.0]","[NO, 13.0]","[BAL, 15.0]","[CIN, 7.0]","[MIA, 4.0]","[JAX, 5.0]","[SF, 10.0]","[CAR, 9.0]","[IND, 12.0]","[PHI, 14.0]","[LAC, 3.0]","[MIN, 2.0]","[TEN, 11.0]","[KC, 8.0]","[TB, 6.0]","[DEN, 16.0]"
36,UrBoiJZ,"[BUF, 8.0]","[ATL, 10.0]","[BAL, 15.0]","[CIN, 16.0]","[MIA, 14.0]","[JAX, 6.0]","[CHI, 7.0]","[CAR, 1.0]","[HOU, 13.0]","[PHI, 2.0]","[LV, 12.0]","[GB, 11.0]","[TEN, 5.0]","[KC, 9.0]","[TB, 4.0]","[SEA, 3.0]"
38,Wild-Bill,"[BUF, 16.0]","[NO, 9.0]","[BAL, 14.0]","[CIN, 2.0]","[MIA, 10.0]","[WAS, 4.0]","[CHI, 3.0]","[CAR, 7.0]","[IND, 15.0]","[PHI, 12.0]","[LAC, 5.0]","[MIN, 1.0]","[TEN, 13.0]","[KC, 11.0]","[DAL, 6.0]","[DEN, 8.0]"
40,Zapp,"[BUF, 1.0]","[NO, 13.0]","[BAL, 14.0]","[CIN, 8.0]","[MIA, 4.0]","[WAS, 5.0]","[SF, 10.0]","[CAR, 6.0]","[IND, 15.0]","[PHI, 16.0]","[LAC, 2.0]","[GB, 7.0]","[TEN, 12.0]","[KC, 9.0]","[TB, 3.0]","[DEN, 11.0]"


In [ ]:
#add spread for home team
#for col in week1.columns:
#    week1[col][1] = [week1[col][1], -week1[col][2][1]]


In [81]:
#function to calculate total points
def totals(df):
    i = 4
    df['Score'] = 0
    while i < 42:
    #print(i)
        for col in df.loc[:, ~df.columns.isin(['Name', 'Score'])]:
            if df[col][0] == df[col][i][0]:
                df['Score'][i] = df['Score'][i] + df[col][i][1]
                #print(week1['Name'][i], ' the ', col, 'was a good game')
            #else:
                #print(week1['Name'][i], "haha, you suck")
        i = i + 2
    return df 

In [82]:
#week1.sort_values(week1['Score'])
#week1[['Name', 'Score']]
week1 = totals(week1)

C:\Users\mtzwa\AppData\Local\Temp\ipykernel_4676\4093087112.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Score'][i] = df['Score'][i] + df[col][i][1]


In [ ]:
week1['Score']

In [73]:
#upset count

def upset_counter(df):
    upset_count = 0
    fav_count = 0
    tie_count = 0
    for col in week1.loc[:, ~week1.columns.isin(['Name', 'Score'])]:
        if week1[col][3] < 0 and week1[col][1] == week1[col][0]: #if away team is favorite and home team won
            upset_count = upset_count + 1
            #print('Home dog pulling it off: ', week1[col][1])
        elif week1[col][3] > 0 and week1[col][2][0] == week1[col][0]: #if home team is favorite and away team won
            upset_count = upset_count + 1
            #print('Road upset alert: ', week1[col][2][0])
        elif week1[col][0] == "TIE":
            tie_count = tie_count + 1
            #print("A friggen tie...  ", week1[col][2])
        else:
            fav_count = fav_count + 1
    
    return upset_count, fav_count, tie_count

In [76]:
upset_count = upset_counter(week1)[0]
fav_count = upset_counter(week1)[1]
tie_count = upset_counter(week1)[2]

print("There were ", upset_count, " upsets in week 1.")



There were  6  upsets in week 1.


In [38]:
#upset_score = 0
#for col in week1.columns:
    #if week1[col][1][1] == week1[0]:
    #    upset_score = upset_score + week1[col][1][1]
    #else if week1[0] == week1[col][1][1]
